In [1]:
import os
from envs.race_strategy_event import RaceEnv
from race_simulation.racesim.src.import_pars import import_pars
from race_simulation.main_racesim import main
from tqdm import tqdm
from collections import defaultdict
import numpy as np
import pandas as pd


In [2]:
directory = "race_simulation/racesim/input/parameters"
MCS_PARS_FILE = 'pars_mcs.ini'

good = []

for entry in tqdm(os.scandir(directory)):
    if entry.is_file:
        filename = entry.name
        if filename != "pars_mcs.ini":
            pars_in, vse_paths = import_pars(use_print=False,
                                            use_vse=False,
                                            race_pars_file=filename,
                                            mcs_pars_file=MCS_PARS_FILE)
            if len(pars_in['event_pars']['fcy_data']['phases']) > 0:
                good.append(filename)

45it [00:00, 143.28it/s]WARNING: Parameter t_gap_overtake_vel is None, continuing with 0.0s!
122it [00:00, 135.93it/s]


In [3]:
from functools import partial
import multiprocessing

def run_main(race):
    # ------------------------------------------------------------------------------------------------------------------
    # USER INPUT -------------------------------------------------------------------------------------------------------
    # ------------------------------------------------------------------------------------------------------------------

    # set race parameter file names
    race_pars_file_ = race
    mcs_pars_file_ = 'pars_mcs.ini'

    # set simulation options
    # use_prob_infl:        activates probabilistic influences within the race simulation -> lap times, pit stop
    #                       durations, race start performance
    # create_rand_events:   activates the random creation of FCY (full course yellow) phases and retirements in the race
    #                       simulation -> they will only be created if the according entries in the parameter file
    #                       contain empty lists, otherwise the file entries are used
    # use_vse:              determines if the VSE (virtual strategy engineer) is used to take tire change decisions
    #                       -> the VSE type is defined in the parameter file (VSE_PARS)
    # no_sim_runs:          number of (valid) races to simulate
    # no_workers:           defines number of workers for multiprocess calculations, 1 for single process, >1 for
    #                       multi-process (you can use print(multiprocessing.cpu_count()) to determine the max. number)
    # use_print:            set if prints to console should be used or not (does not suppress hints/warnings)
    # use_print_result:     set if result should be printed to console or not
    # use_plot:             set if plotting should be used or not
    sim_opts_ = {"use_prob_infl": True,
                    "create_rand_events": False,
                    "use_vse": False,
                    "no_sim_runs": 1,
                    "no_workers": 1,
                    "use_print": False,
                    "use_print_result": False,
                    "use_plot": False}

    # ------------------------------------------------------------------------------------------------------------------
    # SIMULATION CALL --------------------------------------------------------------------------------------------------
    # ------------------------------------------------------------------------------------------------------------------
    main(sim_opts=sim_opts_, race_pars_file=race_pars_file_, mcs_pars_file=mcs_pars_file_)

print("Running simulations...")
t = tqdm(good, desc="Race:", leave=True)

for race in t:
    if "2015" in race or "2016" in race or "2014" in race:
        continue
    t.set_description("Race: %s" % race)
    t.refresh() # to show immediately the update
    race_list = [race] * 100
    with multiprocessing.Pool(multiprocessing.cpu_count()) as pool:
            _ = pool.map(run_main, race_list)
    pool.close()
print("Done")
    

Race: pars_YasMarina_2018.ini: 100%|██████████| 83/83 [02:36<00:00,  1.88s/it]Done



In [3]:
LOGS_FOLDER = "race_simulation/racesim/output"
DATA_FOLDER = "results"
dfs = defaultdict(dict)

for entry in tqdm(os.scandir(LOGS_FOLDER)):
    if os.path.isdir(entry):
        race_name = entry.name
        directory = os.path.join(LOGS_FOLDER, race_name, DATA_FOLDER)
        for entry in os.scandir(directory):
            if entry.is_file:
                filename = entry.name
                filename = filename[:-4]
                tokens = filename.split('_')
                timestamp = tokens.pop(0)
                timestamp += tokens.pop(0)
                try:
                    df = pd.read_csv(entry.path)
                except:
                    print(entry.path)
                    continue
                dfs[tokens[0] + "_" + timestamp][tokens[2]] = df
                dfs[tokens[0] + "_" + timestamp]['year'] = tokens[1]

47it [01:19,  1.69s/it]


In [4]:
# database = pd.DataFrame(columns=['race', 'year', 'lap', 'driver', 'lap_time', 'cumulative_time', 'influence', 'position', 'id'])
database = pd.DataFrame(columns=['race', 'year', 'lap', 'driver', 'cumulative_time', 'position', 'id'])
index = 0
count = 0
laps = [database]
for i, race_id in enumerate(tqdm(dfs)):
    tokens = race_id.split('_')
    timestamp = tokens[1]
    race = tokens[0]
    year = dfs[race_id]['year']

    # lap_times = dfs[race_id]['laptimes']
    # influences = dfs[race_id]['lapinfluences']
    positions = dfs[race_id]['positions']
    race_times = dfs[race_id]['racetimes']

    final_lap = max(race_times['lap'].unique())
    
    columns = list(race_times.columns)
    columns.remove('lap')

    #for lap in lap_times.lap.unique():
        # lap_time_single_lap = lap_times.loc[lap_times['lap'] == lap]
        # influences_single_lap = influences.loc[influences['lap'] == lap]
    positions_single_lap = positions.loc[positions['lap'] == final_lap]
    for col in columns:
        max_race_time = race_times[col].max()
        lap = race_times.loc[race_times[col] == max_race_time, 'lap']
        if len(lap.values)> 0:
            lap = lap.values[0]
        else:
            lap = 1
        index += 1
        new_df = {}
        new_df['id'] = [timestamp]
        new_df['race'] = [race]
        new_df['year'] = [year]
        new_df['lap'] = [lap]
        new_df['driver'] = [col]
        # new_df['lap_time'] = lap_time_single_lap[col].values
        # new_df['influence'] = influences_single_lap[col].values
        new_df['cumulative_time'] = [max_race_time]
        new_df['position'] = positions_single_lap[col].values
        new_df = pd.DataFrame(new_df, index=[index])
        laps.append(new_df)
database = pd.concat(laps)


100%|██████████| 4700/4700 [01:48<00:00, 43.26it/s]


In [5]:
database

,race,year,lap,driver,cumulative_time,position,id
1,Austin,2018,55,VAN,5726.626,12,160863354217687272
2,Austin,2018,8,RIC,823.827,18,160863354217687272
3,Austin,2018,56,VET,5709.743,2,160863354217687272
4,Austin,2018,56,RAI,5703.424,1,160863354217687272
5,Austin,2018,2,GRO,229.656,19,160863354217687272
...,...,...,...,...,...,...,...
93996,YasMarina,2018,55,VER,6093.353,7,160863369790212891
93997,YasMarina,2018,54,SIR,6020.984,13,160863369790212891
93998,YasMarina,2018,55,HAM,6029.542,2,160863369790212891
93999,YasMarina,2018,55,SAI,6115.050,10,160863369790212891


In [6]:
dataframes = []

for race in tqdm(database['race'].unique()):
    race_final = database[database['race'] == race]
    for year in race_final['year'].unique():
        race_year_final = race_final.loc[race_final['year'] == year]
        max_lap = race_year_final['lap'].max()

        # Check if VET has retired
        max_vet = race_year_final.loc[race_year_final['driver'] == 'VET', "lap"].max()
        if max_vet < max_lap - 2:
            print("VET retired or was lapped at", race, year)
        else:
            to_leader = []
            to_front = []
            to_following = []
            classification = []
           
            for ident in race_year_final['id'].unique():
                final_lap = race_year_final.loc[(race_year_final['id'] == ident) & (race_year_final['driver'] == 'VET'), "lap"].values[0]
                if final_lap == max_lap:
                    # Take VET's cumulative time
                    vet_time = race_year_final.loc[(race_year_final['id'] == ident) & (race_year_final['driver'] == 'VET'), 'cumulative_time'].values[0]
                    # Take VET's final position
                    position = race_year_final.loc[(race_year_final['id'] == ident) & (race_year_final['driver'] == 'VET'), 'position'].values[0]
                    classification.append(position)
                    if position != 1:
                        # Compute winner's cumulative time
                        leader_time = race_year_final.loc[(race_year_final['id'] == ident) & (race_year_final['position'] == 1), 'cumulative_time'].values[0]
                        # Compute driver in front's cumulative time
                        if position > 2:
                            in_front_time = race_year_final.loc[(race_year_final['id'] == ident) & (race_year_final['position'] == position - 1), 'cumulative_time'].values[0]
                        if position == 2:
                            in_front_time = leader_time
                    else:
                        in_front_time = leader_time = vet_time

                    if vet_time - in_front_time < 0:
                        print(max_vet, max_lap, position - 1)
                        print(race_year_final.loc[(race_year_final['id'] == ident)])

                    to_front.append(vet_time - in_front_time)
                    to_leader.append(vet_time - leader_time)

                    # Compute following driver's cumulative time
                    following_time = race_year_final.loc[(race_year_final['id'] == ident) & (race_year_final['position'] == position + 1)]
                    if len(following_time) > 0:
                        following_time = following_time['cumulative_time'].values[0]
                    else:
                        following_time = vet_time

                    to_following.append(following_time - vet_time)

            size = len(to_leader)
            dataframes.append(pd.DataFrame({"race": [race] * size, "year" : [year] * size, "position": classification, "to_leader": to_leader, "to_front": to_front, "to_following": to_following}))
            data =  pd.concat(dataframes)     

100%|██████████| 21/21 [00:12<00:00,  1.73it/s]


In [7]:
for year in sorted(data['year'].unique()):
    year_data = data[data['year'] == year]
    for race in sorted(year_data['race'].unique()):
        race_year_data = year_data[year_data['race'] == race]
        print(race, year)
        print(race_year_data.describe())
        print()

    1.00000    0.000000    0.000000      5.872500
50%      1.00000    0.000000    0.000000     13.265500
75%      2.00000    0.400000    0.400000     22.412250
max      5.00000   35.975000   26.815000     51.362000

Catalunya 2017
        position   to_leader    to_front  to_following
count  100.00000  100.000000  100.000000    100.000000
mean     1.92000   24.989470   24.989470     -1.448290
std      0.27266   11.900363   11.900363     23.504267
min      1.00000    0.000000    0.000000    -33.081000
25%      2.00000   18.064000   18.064000    -15.921250
50%      2.00000   27.355500   27.355500     -6.389500
75%      2.00000   32.513250   32.513250      3.419750
max      2.00000   47.523000   47.523000     71.276000

MexicoCity 2017
        position  to_leader   to_front  to_following
count  99.000000  99.000000  99.000000     99.000000
mean    3.929293  50.074071  18.220788      5.315010
std     0.434542  14.234996  10.953152     27.483001
min     3.000000   8.273000   0.400000    -67